In [1]:
import transformers
print(transformers.__version__)
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of GPU

4.53.0
True
1
NVIDIA GeForce RTX 3050 4GB Laptop GPU


In [2]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
print(classifier("I love Hugging Face!"))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998641014099121}]


In [3]:
from datasets import load_dataset
dataset = load_dataset("imdb")
print(dataset["train"][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer("Hello, Hugging Face!", return_tensors="pt")
print(tokens)

{'input_ids': tensor([[  101,  7592,  1010, 17662,  2227,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [6]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model=model_name)
print(classifier("Great tutorial!")) 

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998494386672974}]


In [7]:
dataset = load_dataset("imdb")
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True)
tokenized_dataset = dataset.map(tokenize,batched=True)

In [9]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    fp16=True,
    no_cuda=False
)
trainer = Trainer(model=model,args=training_args,train_dataset=tokenized_dataset["train"])
trainer.train()

Step,Training Loss
500,0.367800
1000,0.307900
1500,0.298400
2000,0.294700
2500,0.263100
3000,0.263600
3500,0.207000
4000,0.181400
4500,0.167800
5000,0.173600


TrainOutput(global_step=9375, training_loss=0.18830241170247397, metrics={'train_runtime': 5197.0295, 'train_samples_per_second': 14.431, 'train_steps_per_second': 1.804, 'total_flos': 9935054899200000.0, 'train_loss': 0.18830241170247397, 'epoch': 3.0})

In [10]:
model.save_pretrained("./fine_tuned_model")

In [14]:
from huggingface_hub import login
login()

In [15]:
model.push_to_hub("my-model")
tokenizer.push_to_hub("my-model")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

c:\Users\amann\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amann\.cache\huggingface\hub\models--aman-nehra--my-model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/aman-nehra/my-model/commit/ca38dd88623eb8bf6ee50c4e145e11c6b78cec7f', commit_message='Upload tokenizer', commit_description='', oid='ca38dd88623eb8bf6ee50c4e145e11c6b78cec7f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aman-nehra/my-model', endpoint='https://huggingface.co', repo_type='model', repo_id='aman-nehra/my-model'), pr_revision=None, pr_num=None)